PRINTED CIRCUIT BOARD (PCB)

A printed circuit board (PCB) mechanically supports and electrically connects electrical or electronic components using conductive tracks, pads and other features etched from one or more sheet layers of copper laminated onto and/or between sheet layers of a non-conductive substrate. Components are generally soldered onto the PCB to both electrically connect and mechanically fasten them to it.
https://en.wikipedia.org/wiki/Printed_circuit_board

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQjGLDZg4tSY1VBfT44mLOP1i2bi9oa6zWq0w&usqp=CAU)nts.com

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from math import log2, ceil
from pathlib import Path
from matplotlib import pyplot as plt 
import cv2
%matplotlib inline
from skimage.measure import compare_ssim
from matplotlib.patches import Rectangle
from keras.preprocessing.image import load_img
from math import sin, cos
from PIL import ImageDraw, Image

#Codes by Martin Piotte https://www.kaggle.com/martinpiotte/bounding-box-data-for-the-whale-flukes

In [ ]:
with open('../input/micropcb-images/train_bboxes.csv') as f: data = f.read().split('\n')[:-1]
len(data) # Number of rows in the dataset

In [ ]:
for line in data[:5]: print(line)

#Codes by Aleksei Tiulpin https://www.kaggle.com/alekseit/simple-bounding-boxes

In [ ]:
def bboxes_pcb(path, img_id, debug=False):
    # You need to input the path (train or test) and the car_id (img_id)
    bboxes = []
    # Let's iterate over all the angles
    for num in range(1, 17):
        # Here we read images i and i+1. 
        # If i==16, we will read the first image
        # To speed up the things, we can scale the images 5 times
        fname1 = os.path.join(path, img_id+ '_{:0>2}.jpg'.format(num))
        fname2 = os.path.join(path, img_id+ '_{:0>2}.jpg'.format((num) % 16+1))
        img_1_orig = cv2.imread(fname1)
        h, w = img_1_orig.shape[0],img_1_orig.shape[1],
        img_1_scaled = cv2.resize(img_1_orig, (w//5, h//5))

        img_2_orig = cv2.imread(fname2)
        h, w = img_2_orig.shape[0],img_2_orig.shape[1],
        img_2_scaled = cv2.resize(img_2_orig, (w//5, h//5))

        if debug:
            plt.figure()
            plt.subplot(121)
            plt.title('Current image [{}]'.format(num))
            plt.imshow(img_1_scaled)
            plt.subplot(122)
            plt.title('Next image [{}]'.format((num) % 16+1))
            plt.imshow(img_2_scaled)
            plt.show()
        
        # As the images differ from each other just a by a small angle of rotation,
        # We can find their difference and draw a boundign box around the obtained image
        img1 = cv2.cvtColor(img_1_scaled, cv2.COLOR_BGR2GRAY)
        img2 = cv2.cvtColor(img_2_scaled, cv2.COLOR_BGR2GRAY)

        # Instead of plain difference, we look for structural similarity
        score, dimg = compare_ssim(img1, img2, full=True)
        dimg = (dimg * 255).astype("uint8")


        thresh = cv2.threshold(dimg, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        if debug:
            plt.figure()
            plt.title('Difference image')
            plt.imshow(dimg>thresh)
            plt.show()        
        
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        ROIS = []
        for c in cnts[1]:
            (x, y, w, h) = cv2.boundingRect(c)
            # We dont want to use too small bounding boxes
            if w*h > img1.shape[0]*img1.shape[1]//9:
                ROIS.append([x, y, x+w, y+h])

        ROIS = np.array(ROIS)

        # Now we will draw a boundig box 
        # around all the bounding boxes (there are outliers)
        x1 = ROIS[:,0].min()
        y1 = ROIS[:,1].min()

        x2 = ROIS[:,2].max()
        y2 = ROIS[:,3].max()

        if debug:
            plt.figure()
            plt.imshow(img_1_orig)
            rect = Rectangle((x1*5, y1*5), (x2-x1)*5, (y2-y1)*5, fill=False, color='red')
            plt.axes().add_patch(rect)
            plt.show()      
        bboxes.append([fname1, x1*5, y1*5, x2*5, y2*5])
    return bboxes

The function didn't work on the next snippet since I got: AttributeError: 'NoneType' object has no attribute 'shape'.

#Codes by Linye Li https://www.kaggle.com/zstusnoopy/visualize-the-location-and-3d-bounding-box-of-car

In [ ]:
train = pd.read_csv('../input/micropcb-images/train_bboxes.csv')
# k is camera instrinsic matrix
k = np.array([[2304.5479, 0,  1686.2379],
           [0, 2305.8757, 1354.9849],
           [0, 0, 1]], dtype=np.float32)

In [ ]:
plt.rcParams["axes.grid"] = False
img_name = train.loc[10]['Image']
pred_string = train.loc[10]['Top']
fig, ax = plt.subplots(figsize=(15, 15))
img = load_img('../input/micropcb-images/train_coded/train_coded/' + img_name)
plt.imshow(img)
plt.show()

In [ ]:
items = pred_string.split(' ')
model_types, yaws, pitches, rolls, xs, ys, zs = [items[i::7] for i in range(7)]

#AttributeError: 'numpy.int64' object has no attribute 'split'

In [ ]:
# convert euler angle to rotation matrix
def euler_to_Rot(yaw, pitch, roll):
    Y = np.array([[cos(yaw), 0, sin(yaw)],
                  [0, 1, 0],
                  [-sin(yaw), 0, cos(yaw)]])
    P = np.array([[1, 0, 0],
                  [0, cos(pitch), -sin(pitch)],
                  [0, sin(pitch), cos(pitch)]])
    R = np.array([[cos(roll), -sin(roll), 0],
                  [sin(roll), cos(roll), 0],
                  [0, 0, 1]])
    return np.dot(Y, np.dot(P, R))

In [ ]:
def draw_line(image, points):
    color = (255, 0, 0)
    cv2.line(image, tuple(points[1][:2]), tuple(points[2][:2]), color, 16)
    cv2.line(image, tuple(points[1][:2]), tuple(points[4][:2]), color, 16)

    cv2.line(image, tuple(points[1][:2]), tuple(points[5][:2]), color, 16)
    cv2.line(image, tuple(points[2][:2]), tuple(points[3][:2]), color, 16)
    cv2.line(image, tuple(points[2][:2]), tuple(points[6][:2]), color, 16)
    cv2.line(image, tuple(points[3][:2]), tuple(points[4][:2]), color, 16)
    cv2.line(image, tuple(points[3][:2]), tuple(points[7][:2]), color, 16)

    cv2.line(image, tuple(points[4][:2]), tuple(points[8][:2]), color, 16)
    cv2.line(image, tuple(points[5][:2]), tuple(points[8][:2]), color, 16)

    cv2.line(image, tuple(points[5][:2]), tuple(points[6][:2]), color, 16)
    cv2.line(image, tuple(points[6][:2]), tuple(points[7][:2]), color, 16)
    cv2.line(image, tuple(points[7][:2]), tuple(points[8][:2]), color, 16)
    return image


def draw_points(image, points):
    image = np.array(image)
    for (p_x, p_y, p_z) in points:
        # print("p_x, p_y", p_x, p_y)
        cv2.circle(image, (p_x, p_y), 5, (255, 0, 0), -1)
    return image

In [ ]:
# image coordinate to world coordinate
def img_cor_2_world_cor():
    x_img, y_img, z_img = img_cor_points[0]
    xc, yc, zc = x_img*z_img, y_img*z_img, z_img
    p_cam = np.array([xc, yc, zc])
    xw, yw, zw = np.dot(np.linalg.inv(k), p_cam)
    print(xw, yw, zw)
    print(x, y, z)

In [ ]:
x_l = 1.02
y_l = 0.80
z_l = 2.31
for yaw, pitch, roll, x, y, z in zip(yaws, pitches, rolls, xs, ys, zs):
    yaw, pitch, roll, x, y, z = [float(x) for x in [yaw, pitch, roll, x, y, z]]
    # I think the pitch and yaw should be exchanged
    yaw, pitch, roll = -pitch, -yaw, -roll
    Rt = np.eye(4)
    t = np.array([x, y, z])
    Rt[:3, 3] = t
    Rt[:3, :3] = euler_to_Rot(yaw, pitch, roll).T
    Rt = Rt[:3, :]
    P = np.array([[0, 0, 0, 1],
                  [x_l, y_l, -z_l, 1],
                  [x_l, y_l, z_l, 1],
                  [-x_l, y_l, z_l, 1],
                  [-x_l, y_l, -z_l, 1],
                  [x_l, -y_l, -z_l, 1],
                  [x_l, -y_l, z_l, 1],
                  [-x_l, -y_l, z_l, 1],
                  [-x_l, -y_l, -z_l, 1]]).T
    img_cor_points = np.dot(k, np.dot(Rt, P))
    img_cor_points = img_cor_points.T
    img_cor_points[:, 0] /= img_cor_points[:, 2]
    img_cor_points[:, 1] /= img_cor_points[:, 2]
    # call this function before chage the dtype
    img_cor_2_world_cor()
    img_cor_points = img_cor_points.astype(int)
    img = draw_points(img, img_cor_points)
    img = draw_line(img, img_cor_points)
    
img = Image.fromarray(img)
plt.imshow(img)
plt.show()

NameError: name 'yaws' is not defined.  Because numpy.int64' object has no attribute 'split'
Therefore, I have NO Image at all.

#Here I just opened the files to read them

In [ ]:
df = pd.read_csv('../input/micropcb-images/train_bboxes.csv', encoding='utf8')
pd.set_option('display.max_columns', None)
df.head()

In [ ]:
df_train = pd.read_csv('../input/micropcb-images/train_angles.csv', encoding='utf8')
pd.set_option('display.max_columns', None)
df_train.head()

In [ ]:
df1 = pd.read_csv('../input/micropcb-images/train_ratio_top_to_bottom.csv', encoding='utf8')
pd.set_option('display.max_columns', None)
df1.head()

In [ ]:
#Code by Olga Belitskaya https://www.kaggle.com/olgabelitskaya/sequential-data/comments
from IPython.display import display,HTML
c1,c2,f1,f2,fs1,fs2=\
'#4251f5','#42a7f5','Akronim','Smokum',30,15
def dhtml(string,fontcolor=c1,font=f1,fontsize=fs1):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family="""\
    +font+"""&effect=3d-float';</style>
    <h1 class='font-effect-3d-float' style='font-family:"""+\
    font+"""; color:"""+fontcolor+"""; font-size:"""+\
    str(fontsize)+"""px;'>%s</h1>"""%string))
    
    
dhtml('Be patient. Marília Prata, @mpwolke was Here.' )